In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
# Import SQLAlchemy modules
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# Connect to sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# Automap database into class
Base = automap_base()
Base.prepare(autoload_with=engine)

In [ ]:
# Check that all the tables were mapped
Base.classes.keys()

In [ ]:
# Save the mapped tables to objects
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Start querying session
session = Session(engine)

In [ ]:
# To find most recent data, arrange dates in descending order and choose first
most_recent_date = session.query(func.max(Measurement.date)).first()
most_recent_date

In [ ]:
# Calculate the date 365 days from the last date in the data set.
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)

# Query to retrieve the data and precipitation scores
results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= prev_year).all()

# Save to dataframe and name columns
df = pd.DataFrame(results, columns=['date', 'precipitation'])

# Sort by date
df = df.sort_values("date")

# Plot with Matplotlib
df.plot(x='date', y='precipitation', rot=45)
plt.xlabel("Date")
plt.ylabel("Inches")

In [ ]:
# Extract summary statistics for precipitation
df.describe()

In [ ]:
# Count number of stations
session.query(func.count(Station.station)).all()

In [ ]:
# Sort stations by activity level
session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

In [ ]:
# Calculate lowest, highest, and average temperature based on most active station.
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
    filter(Measurement.station == 'USC00519281').all()

In [ ]:
# Last 12 months of temperature for the most active station
prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)

results = session.query(Measurement.tobs).\
    filter(Measurement.station == 'USC00519281').\
    filter(Measurement.date >= prev_year).all()
df = pd.DataFrame(results, columns=['tobs'])
df.plot.hist(bins=12)
plt.tight_layout()
plt.xlabel("Temperature")

In [ ]:
# Close session
session.close()